### 1. 임의의 입력 생성하기

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN, LSTM, Bidirectional

In [2]:
train_X = [[0.1, 4.2, 1.5, 1.1, 2.8], [1.0, 3.1, 2.5, 0.7, 1.1], [0.3, 2.1, 1.5, 2.1, 0.1], [2.2, 1.4, 0.5, 0.9, 1.1]]
print(np.shape(train_X))

(4, 5)


* 입력 단어 벡터의 차원은 5(전체 단어갯수)이고, 문장의 길이(sequence)가 4인 경우
* 4번의 시점(timesteps)이 존재하고, 각 시점마다 5차원의 단어 벡터가 입력으로 사용됨

* 그러나 RNN에서는 2D 텐서가 아니라 3D 텐서를 입력 받는다고 했으므로 변경이 필요한데, 배치크기 1을 추가하여 가능

In [3]:
train_X = [[[0.1, 4.2, 1.5, 1.1, 2.8], [1.0, 3.1, 2.5, 0.7, 1.1], [0.3, 2.1, 1.5, 2.1, 0.1], [2.2, 1.4, 0.5, 0.9, 1.1]]]
train_X = np.array(train_X, dtype=np.float32)
print(train_X.shape)

(1, 4, 5)


* (batch_size, timesteps, input_dim)에 해당하는 (1, 4, 5) 크기의 3D 텐서가 생성됨
* batch_size는 샘플 수. RNN이 학습할때 한번에 사용하는 데이터의 양을 말함. 여기에서는 1개 밖에 없음

### 2. SimpleRNN 이해
* 출력값에 대한 이해 
    * return_sequences : 기본값 False. 
    * return_stats : 기본값 False.

In [4]:
### 은닉상태(hidden state) 크기를 3으로 지정하고 두 인자의 값이 False일 때 출력 값
rnn = SimpleRNN(3)
hidden_state = rnn(train_X)

print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))

hidden state : [[-0.99000776  0.7790885  -0.64224684]], shape: (1, 3)


* 은닉상태는 (1, 3) 크기의 텐서로, return_sequences가 False인 경우에는 SimpleRNN은 마지막 은닉상태만 출력
* 만약 return_sequences를 True로 지정하면 모든 시점의 은닉상태 출력함

In [5]:
rnn = SimpleRNN(3, return_sequences=True)
hidden_state = rnn(train_X)

print('hidden state : {}, shape: {}'.format(hidden_state, hidden_state.shape))

hidden state : [[[ 0.9999315  -0.34034526  0.998872  ]
  [ 0.9965539  -0.8557468   0.9934113 ]
  [ 0.9865772   0.68204737 -0.15192412]
  [ 0.9954842  -0.9691426   0.9843184 ]]], shape: (1, 4, 3)


* timesteps가 4이므로 모든 시점에 대해 은닉상태의 값을 출력해서 (1, 4, 3)의 텐서를 출력하게 됨

---
* return_state가 True일 경우, return_sequences가 True/False 여부와 상관없이 마지막 시점의 은닉상태를 출력함. 예를 들어, return_sequences=True이고 return_state=True일 경우, SimpleRNN은 두개 출력을 리턴함

In [6]:
rnn = SimpleRNN(3, return_sequences=True, return_state=True)
hidden_states, last_state = rnn(train_X)

print('hidden states: {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('last state: {}, shape: {}'.format(last_state, last_state.shape))

hidden states: [[[ 0.9142155   0.9667713   0.46622247]
  [ 0.70616555  0.89581203  0.5108151 ]
  [-0.9114635   0.30078605 -0.73716146]
  [ 0.9933788   0.83736104  0.19713785]]], shape: (1, 4, 3)
last state: [[0.9933788  0.83736104 0.19713785]], shape: (1, 3)


* 첫번째 출력은 return_sequences=True로 인한 출력으로 모든 시점의 은닉상태
* 두번째 출력은 return_state=True로 인한 출력으로 마지막 시점의 은닉상태. (1, 4, 3)의 마지막 벡터 값과 일치



* 그렇다면 return_sequences=False이면서 return_state=True이면???  마지막시점만...

In [7]:
rnn = SimpleRNN(3, return_state=True)
hidden_states, last_state = rnn(train_X)
print('hidden states: {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('last state: {}, shape: {}'.format(last_state, last_state.shape))

hidden states: [[0.95848626 0.35654894 0.95581025]], shape: (1, 3)
last state: [[0.95848626 0.35654894 0.95581025]], shape: (1, 3)


두개 출력 모두 마지막 은닉상태만을 출력

### 3. LSTM 이해 
* return_sequences = False,  return_state=True인 경우

In [8]:
lstm = LSTM(3, return_sequences=False, return_state=True)
hidden_states, last_state, last_cell = lstm(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('last state : {}, shape: {}'.format(last_state, last_state.shape))
print('last cell : {}, shape: {}'.format(last_cell, last_cell.shape))

hidden states : [[ 0.27538684  0.27758038 -0.07560135]], shape: (1, 3)
last state : [[ 0.27538684  0.27758038 -0.07560135]], shape: (1, 3)
last cell : [[ 0.8342783   0.37970793 -0.8554363 ]], shape: (1, 3)


* SimpleRNN과 달리 cell 상태 정보가 추가되어 3개의 결과가 반환

---
* return_sequeces=True, return_state=True의 경우

In [9]:
lstm = LSTM(3, return_sequences=True, return_state=True)
hidden_states, last_state, last_cell = lstm(train_X)

print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('last state : {}, shape: {}'.format(last_state, last_state.shape))
print('last cell : {}, shape: {}'.format(last_cell, last_cell.shape))

hidden states : [[[-0.06348065  0.46115878 -0.11123452]
  [-0.09433387  0.68589354 -0.21839036]
  [-0.05105954  0.5660618  -0.3321023 ]
  [ 0.0585146   0.7849436  -0.28598303]]], shape: (1, 4, 3)
last state : [[ 0.0585146   0.7849436  -0.28598303]], shape: (1, 3)
last cell : [[ 0.15670988  1.2803469  -0.5640975 ]], shape: (1, 3)


* 마찬가지로 마지막 은닉상태 값은 전체 은닉상태의 마지막 값과 동일

### 4. Bidirectional(LSTM) 이해
* return_sequences=True와 False의 각 사례에서 은닉상태 값이 어떻게 바뀌는지 비교
* 비교를 위해 출력 은닉상태 값 고정

In [10]:
k_init = tf.keras.initializers.Constant(value=0.1)
b_init = tf.keras.initializers.Constant(value=0)
r_init = tf.keras.initializers.Constant(value=0.1)

* return_sequeces=False, return_state=True의 경우

In [12]:
bilstm = Bidirectional(LSTM(3, return_sequences=False, 
                            return_state=True, kernel_initializer=k_init, 
                            bias_initializer=b_init, 
                            recurrent_initializer=r_init))

hidden_states, forward_h, forward_c, backward_h, backward_c = bilstm(train_X)
print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('forward state : {}, shape: {}'.format(forward_h, forward_h.shape))
print('backward state : {}, shape: {}'.format(backward_h, backward_h.shape))

hidden states : [[0.6303138 0.6303138 0.6303138 0.7038734 0.7038734 0.7038734]], shape: (1, 6)
forward_h : [[0.6303138 0.6303138 0.6303138]], shape: (1, 3)
backward_h : [[0.7038734 0.7038734 0.7038734]], shape: (1, 3)


* return_state가 True인 경우에는 정방향 LSTM의 은닉 상태와 셀 상태, 역방향 LSTM의 은닉 상태와 셀 상태 4가지를 반환
* 첫번째 출력값의 크기가 (1,6)인 것에 주목.. 이는 return_sequences=False인 경우 정방향 LSTM의 마지막 시점의 은닉상태와 역방향 LSTM의 첫번째 시점의 은닉상태가 연결된채 반환되기 때문
* 정방향 LSTM의 마지막 시점의 은닉상태값과 역방향 LSTM 첫번째 은닉상태값을 기억
    * 정방향 LSTM의 마지막 시점의 은닉 상태값 : [0.6303139 0.6303139 0.6303139]
    * 역방향 LSTM의 첫번째 시점의 은닉 상태값 : [0.70387346 0.70387346 0.70387346]
    
* 현재 은닉상태의 값을 고정시켜두었기 때문에 return_seqeunces=True 경우, 출력이 어떻게 바뀌는 비교 가능

In [15]:
bilstm = Bidirectional(LSTM(3, return_sequences=True, return_state=True, 
                            kernel_initializer=k_init, bias_initializer=b_init, 
                            recurrent_initializer=r_init))
hidden_states, forward_h, forward_c, backward_h, backward_c = bilstm(train_X)

In [16]:
print('hidden states : {}, shape: {}'.format(hidden_states, hidden_states.shape))
print('forward state : {}, shape: {}'.format(forward_h, forward_h.shape))
print('backward state : {}, shape: {}'.format(backward_h, backward_h.shape))

hidden states : [[[0.35906473 0.35906473 0.35906473 0.7038734  0.7038734  0.7038734 ]
  [0.55111325 0.55111325 0.55111325 0.58863586 0.58863586 0.58863586]
  [0.59115744 0.59115744 0.59115744 0.3951699  0.3951699  0.3951699 ]
  [0.6303138  0.6303138  0.6303138  0.21942244 0.21942244 0.21942244]]], shape: (1, 4, 6)
forward state : [[0.6303138 0.6303138 0.6303138]], shape: (1, 3)
backward state : [[0.7038734 0.7038734 0.7038734]], shape: (1, 3)


* 모든 시점에서 은닉상태가 출력되는데, 역방향 LSTM의 첫번째 시점의 은닉상태는 더이상 정방향 LSTM의 마지막 은닉상태와 연결되는 것이 아니라, 정방향 LSTM의 첫번째 시점의 은닉상태만 연결